<a href="https://colab.research.google.com/github/ananyaem/UPSC-LLM/blob/main/RAG_LangChain_UPSC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 -q install faiss-cpu pandas requests langchain langchain-cohere langchain-community

In [ ]:
!wget "https://docs.google.com/spreadsheets/u/7/d/1JNgL-tDuU_R6BjvBgEzUjTJwEDTpqv3zK08KZ6Ab85Q/export?format=csv&id=1JNgL-tDuU_R6BjvBgEzUjTJwEDTpqv3zK08KZ6Ab85Q&gid=1175330818" -O data.csv

--2024-05-23 12:04:21--  https://docs.google.com/spreadsheets/u/7/d/1JNgL-tDuU_R6BjvBgEzUjTJwEDTpqv3zK08KZ6Ab85Q/export?format=csv&id=1JNgL-tDuU_R6BjvBgEzUjTJwEDTpqv3zK08KZ6Ab85Q&gid=1175330818
Resolving docs.google.com (docs.google.com)... 172.253.63.100, 172.253.63.113, 172.253.63.101, ...
Connecting to docs.google.com (docs.google.com)|172.253.63.100|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-08-9g-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/i81138rrhh2cn2vo9p2e7n4ufs/1716465860000/115055468268182144634/*/1JNgL-tDuU_R6BjvBgEzUjTJwEDTpqv3zK08KZ6Ab85Q?format=csv&id=1JNgL-tDuU_R6BjvBgEzUjTJwEDTpqv3zK08KZ6Ab85Q&gid=1175330818 [following]
--2024-05-23 12:04:21--  https://doc-08-9g-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/i81138rrhh2cn2vo9p2e7n4ufs/1716465860000/115055468268182144634/*/1JNgL-tDuU_R6BjvBgEzUjTJwEDTpqv3zK08KZ6Ab85Q?format=csv&id=1JNgL-tDuU_R6BjvBgEzUjTJwEDTpqv3zK08KZ6Ab85Q&

In [ ]:
!head data.csv

Date,Read?,Link,Chars,Title,Original,% match,1920,362,18.85%
2023-12-02,Y,https://epaper.thehindu.com/ccidist-ws/th/th_delhi/issues/61803/OPS/GRKC3E4L8.1+GQ0C3F8IK.1.html,821,"RBI, BoE agree to exchange information on CCIL","RBI, BoE agree to exchange information on CCIL:",90.1,,,
2023-12-02,Y,https://epaper.thehindu.com/ccidist-ws/th/th_delhi/issues/61803/OPS/GRKC3E4KU.1+GQ0C3F8ON.1.html,1088,India offers to host 2028 climate meet,India offers to host 2028 climate meet:,89.8,,,
2023-12-02,Y,https://epaper.thehindu.com/ccidist-ws/th/th_delhi/issues/61803/OPS/GRKC3E4KU.1+GQ0C3F8P1.1.html,1204,"T.N. Governor cannot refer re-enacted Bills to President, says SC",T.N: Governor cannot refer re-enacted Bills to President; says SC:,87.4,,,
2023-12-02,Y,https://epaper.thehindu.com/ccidist-ws/th/th_delhi/issues/61803/OPS/GRKC3E4KU.1+GQ0C3F8OT.1.html,1460,GST revenues rise to â¹1.68 lakh cr. in November; fastest growth this fiscal,GST revenues rise to <1.68 lakh cr. in November; fastest growth t

In [ ]:
import pandas as pd
import requests


In [ ]:
df = pd.read_csv("data.csv")

In [ ]:
df.head()

,Date,Read?,Link,Chars,Title,Original,% match,1920,362,18.85%
0,2023-12-02,Y,https://epaper.thehindu.com/ccidist-ws/th/th_d...,821,"RBI, BoE agree to exchange information on CCIL","RBI, BoE agree to exchange information on CCIL:",90.1,NaN,NaN,NaN
1,2023-12-02,Y,https://epaper.thehindu.com/ccidist-ws/th/th_d...,1088,India offers to host 2028 climate meet,India offers to host 2028 climate meet:,89.8,NaN,NaN,NaN
2,2023-12-02,Y,https://epaper.thehindu.com/ccidist-ws/th/th_d...,1204,T.N. Governor cannot refer re-enacted Bills to...,T.N: Governor cannot refer re-enacted Bills to...,87.4,NaN,NaN,NaN
3,2023-12-02,Y,https://epaper.thehindu.com/ccidist-ws/th/th_d...,1460,GST revenues rise to â¹1.68 lakh cr. in Novem...,GST revenues rise to <1.68 lakh cr. in Novembe...,88.2,NaN,NaN,NaN
4,2023-12-02,Y,https://epaper.thehindu.com/ccidist-ws/th/th_d...,1610,Manufacturing PMI rebounds in Nov.,Manufacturing PMI rebounds in Nov::,86.8,NaN,NaN,NaN


In [ ]:
links = df['Link'].dropna().tolist()
len(links)

1920

In [ ]:
from datetime import datetime

In [ ]:
!pip3  install --upgrade --quiet  nest_asyncio

# fixes a bug with asyncio and jupyter
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(links,requests_per_second=100)
a= datetime.now()
data = loader.aload()
b= datetime.now()
print(b-a)

Fetching pages: 100%|##########| 1920/1920 [00:31<00:00, 60.45it/s]


0:00:38.468545


In [ ]:
COHERE_API_KEY= "GRX6APWCBku4np94O13i5CHrODC6vYcFDu7uhFdF"

In [ ]:
from langchain_cohere import ChatCohere

llm = ChatCohere(cohere_api_key=COHERE_API_KEY)

In [ ]:
llm.invoke("hello anu?")

AIMessage(content='Hello! How can I help you today?', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'cd60ea64-87c1-4fe5-ab0a-9ff3d2c8943e', 'token_count': {'input_tokens': 70, 'output_tokens': 9}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'cd60ea64-87c1-4fe5-ab0a-9ff3d2c8943e', 'token_count': {'input_tokens': 70, 'output_tokens': 9}}, id='run-9107168b-8a04-4f3d-b04e-16cb082fb71d-0')

In [ ]:
from langchain_cohere.embeddings import CohereEmbeddings

embeddings = CohereEmbeddings(cohere_api_key=COHERE_API_KEY)

In [ ]:
from langchain_community.vectorstores import FAISS
vector = FAISS.from_documents(data, embeddings)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({"input": """

Meira Paibis, a powerful civil society activists recently seen in the news are from which of the following Indian state?

	A.
Kerala

	B.
Gujarat

	C.
Arunachal Pradesh

	D.
Manipur


"""})
print(response["answer"])

D. Manipur


In [ ]:
vector.save_local("faiss_index")
new_db = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field, conlist, validator


class Answer(BaseModel):
  options: conlist(str, min_items=1, max_items=4) = Field(description="The correct options from (A/B/C/D). Some questions may have multiple correct options.")

  @validator('options', each_item=True)
  def check_options(cls, v):
    if len(set(v)) != len(v):
      raise ValueError("Options should be unique")
    if v not in {"A", "B", "C", "D"}:
      raise ValueError("Each item must be one of 'A', 'B', 'C', or 'D'")
    return v

In [ ]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

def inspect(stage):
  def inspector(state):
    """Print the state passed between Runnables in a langchain and pass it on"""
    print("Stage is", stage)
    print("State is", state)
    return state
  return inspector

def get_context(state):
  meta = []
  for doc in state['context']:
    meta.append(doc.metadata)
    print(doc.metadata)
  state['meta'] = meta
  return state

chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | RunnableLambda(get_context)
    | prompt
    | llm.with_structured_output(Answer)
)

In [ ]:
x = chain.invoke("""
The Nari Shakti Vandan Adhiniyam, 2023,
that provides for33% women reservation of
seats in the Lok Sabha and State Legislative
Assemblies, is based on the principle of:
(a) equality of opportunity
(b) equality before law
(c) affirmative action
(d) equal rights
""")
print(x.options)

{'source': 'https://epaper.thehindu.com/ccidist-ws/th/th_delhi/issues/52554/OPS/G3FBPBCPP.1+GVLBPBGBS.1.html', 'title': 'Legislating change', 'language': 'No language found.'}
{'source': 'https://epaper.thehindu.com/ccidist-ws/th/th_delhi/issues/53484/OPS/GQ1BQ9NIP.1+GSVBQBG1O.1.html', 'title': 'From women’s reservation to gender equality', 'language': 'No language found.'}
{'source': 'https://epaper.thehindu.com/ccidist-ws/th/th_delhi/issues/55332/OPS/GT9BS7FMQ.1+GDHBS9L6S.1.html', 'title': 'Women’s quota — rhetoric and reality', 'language': 'No language found.'}
{'source': 'https://epaper.thehindu.com/ccidist-ws/th/th_delhi/issues/59139/OPS/GFGC0DGIP.1+GGUC0DI4T.1.html', 'title': 'Nagaland nod for 33% quota for women in urban local bodies', 'language': 'No language found.'}
['C']
